# Get Embeddings

In [2]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set up Azure OpenAI

In [3]:
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = "https://tutorial-openai-01-2023.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

True

## Load Data

In [4]:
import pandas as pd

df_orig = pd.read_csv("../data/bbc-news-data.csv", delimiter='\t')
df = df_orig.copy()
df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


## Deploy a model

In [5]:
# list models deployed with embeddings capability
deployment_id = None
result = openai.Deployment.list()

for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])
    if model["capabilities"]["embeddings"] != True:
        continue
    
    deployment_id = deployment["id"]
    break

# if not model deployed, deploy one
if not deployment_id:
    print('No deployment with status: succeeded found.')
    model = "text-similarity-davinci-001"

    # Now let's create the deployment
    print(f'Creating a new deployment with model: {model}')
    result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard"})
    deployment_id = result["id"]
    print(f'Successfully created {model} with deployment_id {deployment_id}')
else:
    print(f'Found a succeeded deployment that supports embeddings with id: {deployment_id}.')

Found a succeeded deployment that supports embeddings with id: deployment-89153abdfa934e1580296dbee586239b.


## Get Embeddings
ref: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/tutorials/embeddings?tabs=bash

In [6]:
embedding = openai.Embedding.create(
    input="Your text goes here",
    deployment_id=deployment_id)

# embedding = openai.Embedding.create(
#     input="Your text goes here",
#     model="text-similarity-davinci-001") # <-- this will fail

#embedding
embedding["data"][0]["embedding"]
len(embedding["data"][0]["embedding"])

[-0.00945579819381237,
 0.00753916846588254,
 -0.01116229873150587,
 -0.006711388006806374,
 -0.01154435146600008,
 0.020987413823604584,
 -0.0024196654558181763,
 0.003413001773878932,
 0.005148156546056271,
 -0.005272323731333017,
 -0.00040075709694065154,
 0.0003006673068739474,
 0.0006634181481786072,
 -0.006087368819862604,
 -0.0009336406947113574,
 -0.0005034336936660111,
 -0.007424552459269762,
 -0.004568710457533598,
 -0.00781297218054533,
 0.012970680370926857,
 -0.011614394374191761,
 -0.0024117061402648687,
 0.020414335653185844,
 -0.0019803051836788654,
 0.006953354459255934,
 0.005469717551022768,
 0.014454317279160023,
 0.002207944868132472,
 -0.007017029915004969,
 -0.00614149309694767,
 -0.006705020554363728,
 0.004301273729652166,
 -0.002295498503372073,
 -0.0036135793197900057,
 -0.011296017095446587,
 -0.0005217403522692621,
 0.013053459115326405,
 0.00376958423294127,
 -0.0037345627788454294,
 0.01039182674139738,
 -0.013613802380859852,
 -0.002865393413230777,
 -0.

12288

In [11]:
df['embedding'] = ''

for i in range(len(df)):    
#for i in range(760,765):
    try:
        embedding = openai.Embedding.create(input=df['content'][i], deployment_id=deployment_id)
        #len(embedding["data"][0]["embedding"])
        df['embedding'][i] = embedding['data'][0]['embedding']
    except Exception as err:
        i
        print(f"Unexpected {err=}, {type(err)=}")
        #raise

762

Unexpected err=InvalidRequestError(message="This model's maximum context length is 2046 tokens, however you requested 4300 tokens (4300 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", param=None, code=None, http_status=400, request_id=None), type(err)=<class 'openai.error.InvalidRequestError'>


765

Unexpected err=InvalidRequestError(message="This model's maximum context length is 2046 tokens, however you requested 2940 tokens (2940 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", param=None, code=None, http_status=400, request_id=None), type(err)=<class 'openai.error.InvalidRequestError'>


862

...

Unexpected err=InvalidRequestError(message="This model's maximum context length is 2046 tokens, however you requested 3842 tokens (3842 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", param=None, code=None, http_status=400, request_id=None), type(err)=<class 'openai.error.InvalidRequestError'>


1683

Unexpected err=InvalidRequestError(message="This model's maximum context length is 2046 tokens, however you requested 2059 tokens (2059 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", param=None, code=None, http_status=400, request_id=None), type(err)=<class 'openai.error.InvalidRequestError'>


2224

Unexpected err=InvalidRequestError(message="This model's maximum context length is 2046 tokens, however you requested 3545 tokens (3545 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", param=None, code=None, http_status=400, request_id=None), type(err)=<class 'openai.error.InvalidRequestError'>


## Save data

In [12]:
#df.to_csv("../data/bbc-news-data-embedding.csv", sep='\t')